In [1]:
import pandas as pd

students_df = pd.read_csv("studentCourse.csv")
courses_temp = pd.read_csv("courses.csv", index_col="Course Code")
courses_df = courses_temp[~courses_temp.index.duplicated(keep='first')]
teachers_df = pd.read_csv("teachers.csv")



POPULATION GENERATION

In [2]:
import numpy as np
import pandas as pd

def select_student_for_exam(course_code, time_slot, scheduled_students, students_df):
    available_students = set(students_df[students_df["Course Code"] == course_code]["Student Name"])

    return available_students


students_dict = {}

def create_chromosome(classrooms_list, exam_duration, break_duration=30):
    chromosome_df = pd.DataFrame(columns=["Day", "Time", "Course", "Classroom", "Teacher"])

    courses_list = list(courses_df.index)
    np.random.shuffle(courses_list)

    teachers_list = list(teachers_df["Names"])
    np.random.shuffle(teachers_list)

    days_list = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

    time_slots = []
    start_time = pd.Timestamp("09:00")
    end_day_time = pd.Timestamp("17:00")

    while start_time + pd.Timedelta(hours=exam_duration) <= end_day_time:
        end_time = start_time + pd.Timedelta(hours=exam_duration)
        time_slots.append((start_time.strftime("%H:%M"), end_time.strftime("%H:%M")))
        start_time = end_time + pd.Timedelta(minutes=break_duration)

    scheduled_students = {time_slot: set() for time_slot in time_slots}

    for day in days_list:
        if not courses_list:
            break
        for start_time, end_time in time_slots:
            if not courses_list:
                break

            for classroom in classrooms_list:
                if not courses_list:
                    break
                course = courses_list.pop(0)

                teacher = teachers_list.pop(0)

                while True:

                    students = select_student_for_exam(course, (start_time, end_time), scheduled_students, students_df)


                    if students in scheduled_students[(start_time, end_time)]:
                        if len(students_df[students_df["Course Code"] == course]) == len(scheduled_students[(start_time, end_time)]):
                            break
                        np.random.shuffle(courses_list)
                        course = courses_list[0]
                        # print("*")
                    else:
                        for student in students:
                            scheduled_students[(start_time, end_time)].add(student)

                        students_dict[course] = (students)

                        break

                chromosome_df.loc[len(chromosome_df)] = [day, f"{start_time} - {end_time}", course, classroom, teacher]

    return chromosome_df




FITNESS FUNCTIONS

In [3]:
def ff1(chromosome):
    fitness = 10

    temp = chromosome["Course"].unique()
    No_of_courses = len(list(temp))
    if No_of_courses < 23:
        fitness -= 1

    # if fitness==10:
    #     print("An exam will be scheduled for each course.: SUCCESS")
    # else:
    #     print("An exam will be scheduled for each course.: FAIL")

    return fitness


# A teacher cannot invigilate two exams at the same time.
def ff2(chromosome):
    temp = chromosome.groupby(["Day", "Time"])["Teacher"].unique()
    fitness = 10
    counter = 0
    for i in range(len(temp)):
        l = temp.iloc[i]
        for j in range(len(l)):
            if j in l:
                counter += 1        #check if teacher repeating within same time slot.

    # if counter<=1:
    #     print("A teacher cannot invigilate two exams at same time: SUCCESS ")
    # else:
    #     fitness -= 1
    #     print("A teacher cannot invigilate two exams at same time: FAIL")

    return fitness


# A teacher cannot invigilate two exams in a row
def ff3(chromosome):
    temp = chromosome.groupby(["Day", "Time"])["Teacher"].unique()
    fitness = 10
    for i in range(len(temp)-1):
        l1 = temp.iloc[i]
        l2 =  temp.iloc[i+1]
        for j in l2:
            if j in l1:
                fitness -= 1


    # if fitness==10:
    #     print("A teacher cannot invigilate two exams in a row: SUCCESS ")
    # else:
    #     print("A teacher cannot invigilate two exams in a row: FAIL")

    return fitness


# A student cannot give more than 1 exam at a time.
def ff4(chromosome):
    fitness = 10

    for day, time_slot_group in chromosome.groupby(["Day", "Time"]):
        students_in_time_slot = set()
        for _, time_slot_info in time_slot_group.iterrows():
            course = time_slot_info["Course"]
            students = students_dict[course]
            for student in students:
                if student in students_in_time_slot:
                    fitness -= 1
                else:
                    students_in_time_slot.add(student)

    # print(len(students_in_time_slot))

    # if fitness==10:
    #     print("A student cannot give more than 1 exam at a time: SUCCESS")
    # else:
    #     print("A student cannot give more than 1 exam at a time: FAIL")

    return fitness


# All students and teachers shall be given a break on Friday from 1-2.
def sc_ff1(chromosome):
    fitness = 10

    filtered_df = chromosome[chromosome["Day"]=="Friday"]

    break_start_time = pd.Timestamp("13:00")
    break_end_time = pd.Timestamp("14:00")

    for _, exam in filtered_df.iterrows():
        start, end = exam["Time"].split("-")
        start = pd.Timestamp(start)
        end = pd.Timestamp(end)

        if (start<break_end_time and end>break_start_time):
            fitness -= 1

    # if fitness == 10:
    #     print("Friday break constraint satisfied: SUCCESS")
    # else:
    #     print("Friday break constraint violated: FAIL")

    return fitness


# A student shall not give more than 1 exam consecutively.
def sc_ff2(chromosome):
    fitness = 10

    for i in range(0, len(chromosome)-1):
        previous_course = chromosome.at[i, "Course"]
        current_course = chromosome.at[i + 1, "Course"]
        previous_students = students_dict[previous_course]
        current_students = students_dict[current_course]

        for student in current_students:
            if (student in previous_students):
                fitness -= 1

    # if fitness==10:
    #     print("A student shall not give more than 1 exam consecutively: SUCCESS")
    # else:
    #     print("A student shall not give more than 1 exam consecutively: FAIL")

    return fitness


# MG course exam held before their CS course exam.
def sc_ff3(chromosome):
    fitness = 10

    student_course_timing = {}

    for index, row in chromosome.iterrows():
        course = row["Course"]
        day_time = row["Time"]
        students = students_dict.get(course, set())

        for student in students:
            if "MG" in course:
                if student not in student_course_timing:
                    student_course_timing[student] = {"earliest_MG": day_time, "latest_CS": None}
                else:
                    if student_course_timing[student]["earliest_MG"] is None or day_time < student_course_timing[student]["earliest_MG"]:
                        student_course_timing[student]["earliest_MG"] = day_time

            elif "CS" in course:
                if student not in student_course_timing:
                    student_course_timing[student] = {"earliest_MG": None, "latest_CS": day_time}
                else:
                    if student_course_timing[student]["latest_CS"] is None or day_time > student_course_timing[student]["latest_CS"]:
                        student_course_timing[student]["latest_CS"] = day_time

    for student, times in student_course_timing.items():
        if (times["earliest_MG"]) and (times["latest_CS"]) and (times["earliest_MG"] >= times["latest_CS"]):
            fitness -= 1

    # if fitness == 10:
    #     print("MG course exam held before their CS exam: SUCCESS")
    # else:
    #     print("MG course exam held before their CS exam: FAIL")

    return fitness



def calculate_fitness(chromosome):
    fitness = 0
    fulfilled_constraints = []

    # Check fulfilled hard and soft constraints
    fulfilled_constraints.append("Exam will not be held on weekends.")
    fulfilled_constraints.append("Each exam must be held between 9 am and 5 pm.")
    if ff1(chromosome) == 10:
        fulfilled_constraints.append("An exam will be scheduled for each course.")
    if ff2(chromosome) == 10:
        fulfilled_constraints.append("A teacher cannot invigilate two exams at the same time.")
    if ff3(chromosome) == 10:
        fulfilled_constraints.append("A teacher cannot invigilate two exams in a row.")
    if ff4(chromosome) == 10:
        fulfilled_constraints.append("A student cannot give more than 1 exam at a time.")
    else:
        fulfilled_constraints.append("A student cannot give more than 1 exam at a time.")
    if sc_ff1(chromosome) == 10:
        fulfilled_constraints.append("All students and teachers shall be given a break on Friday from 1-2.")
    if sc_ff2(chromosome) == 10:
        fulfilled_constraints.append("A student shall not give more than 1 exam consecutively.")
    if sc_ff3(chromosome) == 10:
        fulfilled_constraints.append("MG course exam held before their CS course exam.")
    fulfilled_constraints.append("Two hours of break in the week")

    # Hard constraints
    fitness += ff1(chromosome)
    fitness += ff2(chromosome)
    fitness += ff3(chromosome)
    fitness += ff4(chromosome)

    # Soft constraints
    fitness += sc_ff1(chromosome)
    fitness += sc_ff2(chromosome)
    fitness += sc_ff3(chromosome)


    return fitness, fulfilled_constraints



SELECTION

In [4]:
import random

def roulette_wheel_selection(population, fitnesses):
    total_fitness = sum(fitnesses)
    relative_fitness = [f / total_fitness for f in fitnesses]

    cumulative_prob = [sum(relative_fitness[:i+1]) for i in range(len(relative_fitness))]

    selected = []
    for _ in range(len(population)):
        r = random.random()
        for i, prob in enumerate(cumulative_prob):
            if r <= prob:
                selected.append(population[i])
                break

    return selected


CROSSOVER

In [5]:

def partial_ordered_crossover(p1, p2):
    # Make copies of parents
    offspring1 = p1.copy()
    offspring2 = p2.copy()

    rang = len(p1)/2
    No_of_courses = np.random.choice((1,int(rang)))

    # Randomly select some courses for crossover
    courses_to_crossover = np.random.choice(p1["Course"], size=No_of_courses, replace=False)

    # Find the indices of selected courses in offspring1 and offspring2
    indices_offspring1 = [offspring1.index[offspring1["Course"] == course].tolist()[0] for course in courses_to_crossover]
    indices_offspring2 = [offspring2.index[offspring2["Course"] == course].tolist()[0] for course in courses_to_crossover]
    indices_offspring1.sort()
    indices_offspring2.sort()

    for k in range(No_of_courses):
        i = indices_offspring1[k]
        j = indices_offspring2[k]
        offspring1.iloc[i]["Course"] = p2.iloc[j]["Course"]
        offspring2.iloc[j]["Course"] = p1.iloc[i]["Course"]


    return offspring1, offspring2




MUTATION

In [6]:
# This mutation involves swapping the values of two randomly selected genes within a chromosome.
def mutation(chromosome):
    mutated_chromosome = chromosome.copy()
    mutation_rate = 0.35

    if np.random.rand() <= mutation_rate:
        # print("TRUE")
        # for i in range(2):
        j, k = np.random.choice(len(chromosome), size=2, replace=True)

        mutated_chromosome.iloc[j]["Course"] = chromosome.iloc[k]["Course"]
        mutated_chromosome.iloc[k]["Course"] = chromosome.iloc[j]["Course"]

    return mutated_chromosome


MAIN

In [7]:
max_gen = 20
pop_size = 10

classrooms_input = input("Enter the list of classrooms (separated by commas): ").strip().split(",")
exam_duration_input = int(input("Enter the duration of exams (in hours): "))
break_duration_input = int(input("Enter the duration of the break between exams (in minutes): "))

population = [create_chromosome(classrooms_input, exam_duration_input, break_duration_input) for _ in range(pop_size)]

best_chromosome = None
best_fitness = float('-inf')  # Initialize best fitness with negative infinity

for gen in range(max_gen):
    fitnesses = [calculate_fitness(chromosome)[0] for chromosome in population]

    new_population = []
    while len(new_population) < pop_size - 1:  # Keep space for the best chromosome
        selected_population = roulette_wheel_selection(population, fitnesses)
        best1, best2, worst1, worst2 = selected_population[:4]

        offspring1, offspring2 = partial_ordered_crossover(worst1, worst2)

        of1_fitness, of1_constraints = calculate_fitness(offspring1)
        of2_fitness, of2_constraints = calculate_fitness(offspring2)

        if of1_fitness < of2_fitness:
            offspring1 = mutation(offspring1)
            new_population.append(offspring1)
        else:
            offspring2 = mutation(offspring2)
            new_population.append(offspring2)

    if best_chromosome is not None:
        new_population.append(best_chromosome)

    fitnesses = [calculate_fitness(chromosome) for chromosome in new_population]

    best_chromosome_index = fitnesses.index(max(fitnesses))
    best_chromosome = new_population[best_chromosome_index]
    best_fitness = calculate_fitness(best_chromosome)[0]  # Get the fitness score only

    population[-1] = best_chromosome

    print("*" * 70)
    print(f"Generation {gen}: Fitness = {best_fitness}")
    print("Fulfilled Constraints:")
    for constraint in calculate_fitness(best_chromosome)[1]:  # Get the fulfilled constraints
        print("- ", constraint)
    print("-" * 70)
    print(best_chromosome)
    print("-" * 70)


**********************************************************************
Generation 0: Fitness = 35
Fulfilled Constraints:
-  Exam will not be held on weekends.
-  Each exam must be held between 9 am and 5 pm.
-  An exam will be scheduled for each course.
-  A teacher cannot invigilate two exams at the same time.
-  A teacher cannot invigilate two exams in a row.
-  A student cannot give more than 1 exam at a time.
-  All students and teachers shall be given a break on Friday from 1-2.
-  Two hours of break in the week
----------------------------------------------------------------------
          Day           Time  Course Classroom            Teacher
0      Monday  09:00 - 11:00   CS217        C1      Ameen Chilwan
1      Monday  09:00 - 11:00   CS220        C2       Sadia Nauman
2      Monday  09:00 - 11:00   CS218        C3      Maheen Arshad
3      Monday  11:10 - 13:10   CS328        C1         Ejaz Ahmed
4      Monday  11:10 - 13:10   CS118        C2     Zeeshan Qaiser
5      Mon